In [56]:
# PennyLane imports
import pennylane as qml

from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

# General imports
import os
import json
import numpy as np
from datetime import datetime, timedelta

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from qiskit_ibm_runtime import QiskitRuntimeService, Session


In [ ]:
#service = QiskitRuntimeService(
#    channel='ibm_quantum',
#    instance='ibm-q/open/main',
#    token=''
#)


In [ ]:
#QiskitRuntimeService.save_account(channel='ibm_quantum', instance='ibm-q/open/main', token='', overwrite=True)

In [57]:
service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')
#session = Session(backend=backend)

In [58]:
potential = 'AHO'
cut_off = 8
shots = 2

In [59]:
H = calculate_Hamiltonian(cut_off, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

HP = qml.pauli_decompose(H)

In [60]:
dev = qml.device('qiskit.remote', wires=num_qubits, backend=backend, shots=shots)
device_time = timedelta()

# Define the cost function
@qml.qnode(dev)
def cost_function(params):

    global device_time
    start = datetime.now()

    qml.RY(params[0], wires=[0])
    qml.RY(params[1], wires=[num_qubits-2])

    end = datetime.now()
    device_time += (end - start)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))#qml.expval(HP)
    

In [61]:
# VQE
vqe_start = datetime.now()

#variables
num_vqe_runs = 1
max_iter = 30
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 5

# Generate Halton sequence
num_dimensions = 2
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#data arrays
energies = []
x_values = []
success = []
run_times = []
num_iters = []
num_evaluations = []

#Optimizer
bounds = [(0, 2 * np.pi) for _ in range(2)]

with Session(backend=backend) as session:
    for i in range(num_vqe_runs):

        run_start = datetime.now()

        if i % 10 == 0:
            print(f"Run: {i}")

        # Differential Evolution optimization
        res = differential_evolution(cost_function,
                                    bounds,
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples
                                    )
        
        if res.success == False:
            print("Not converged")

        energies.append(res.fun)
        x_values.append(res.x)
        success.append(res.success)
        num_iters.append(res.nit)
        num_evaluations.append(res.nfev)

        run_end = datetime.now()
        run_time = run_end - run_start
        run_times.append(run_time)

    total_run_time = sum(run_times, timedelta())
    vqe_end = datetime.now()
    vqe_time = vqe_end - vqe_start

Run: 0


c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\pennylane_qiskit\qiskit_device.py:224: UserWarning: The observable measured Hermitian(array([[  3.4375    +0.j,   0.        +0.j,   7.15945616+0.j,
          0.        +0.j,   5.81753814+0.j,   0.        +0.j,
          1.67705098+0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j],
       [  0.        +0.j,  14.5625    +0.j,   0.        +0.j,
         24.0356181 +0.j,   0.        +0.j,  17.11632992+0.j,
          0.        +0.j,   4.43705984+0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j],
       [  7.15945616+0.j,   0.        +0.j,  39.9375    +0.j,
          0.        +0.j,  56.9411703 +0.j,   0.        +0.j,
         36.7614778 +0

Not converged


In [62]:
res

             message: Maximum number of iterations has been exceeded.
             success: False
                 fun: 0.03201011000923269
                   x: [ 3.552e+00  4.769e+00]
                 nit: 30
                nfev: 218
          population: [[ 3.552e+00  4.769e+00]
                       [ 1.738e+00  4.556e+00]
                       [ 3.184e+00  4.719e+00]
                       [ 3.408e+00  4.722e+00]
                       [ 3.838e+00  4.728e+00]]
 population_energies: [ 3.201e-02  8.561e-01  8.561e-01  2.021e+00
                        8.561e-01]

In [63]:
# Save run
run = {
    "starttime": str(vqe_start),
    "potential": potential,
    "cutoff": cut_off,
    "exact_eigenvalues": [x.real.tolist() for x in eigenvalues],
    "ansatz": "StronglyEntanglingLayers-1layer",
    "num_VQE": num_vqe_runs,
    "shots": shots,
    "Optimizer": {
        "name": "differential_evolution",
        "bounds": "[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]",
        "maxiter": max_iter,
        "tolerance": tol,
        "abs_tolerance": abs_tol,
        "strategy": strategy,
        "popsize": popsize,
        'init': 'scaled_samples',
    },
    "results": energies,
    "params": [x.tolist() for x in x_values],
    "num_iters": num_iters,
    "num_evaluations": num_evaluations,
    "success": np.array(success, dtype=bool).tolist(),
    "run_times": [str(x) for x in run_times],
    "parallel_run_time": str(vqe_time),
    "total_VQE_time": str(total_run_time),
    "total_device_time": str(device_time),
    "Backend": backend.name
}

run

{'starttime': '2025-02-19 11:31:18.273819',
 'potential': 'AHO',
 'cutoff': 8,
 'exact_eigenvalues': [0.03201011000921177,
  1.6801555245826338,
  1.8335255754908608,
  3.963373907862429,
  4.009854623022724,
  4.03692369084855,
  8.262093942430328,
  8.776474914447268,
  42.528080920192636,
  43.43333536006073,
  55.306689599326106,
  56.206325851357306,
  381.87853888129337,
  382.8212806220673,
  408.6442927673947,
  409.5870437096132],
 'ansatz': 'StronglyEntanglingLayers-1layer',
 'num_VQE': 1,
 'shots': 2,
 'Optimizer': {'name': 'differential_evolution',
  'bounds': '[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]',
  'maxiter': 30,
  'tolerance': 0.001,
  'abs_tolerance': 0.001,
  'strategy': 'randtobest1bin',
  'popsize': 5,
  'init': 'scaled_samples'},
 'results': [np.float64(0.03201011000923269)],
 'params': [[3.551511025692399, 4.769269198764862]],
 'num_iters': [30],
 'num_evaluations': [218],
 'success': [False],
 'run_times': ['0:33:47.632040'],
 'parallel_run_time

In [64]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\Differential Evolution\Real Device\Files\\"
path = base_path + "{}_{}.json".format(potential, cut_off)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)